In [1]:
# Generate the video for the env with fake action

In [3]:
import os
import torch
import gymnasium as gym

from omegaconf import OmegaConf

In [4]:
import torchdriveenv
from torchdriveenv.env_utils import construct_env_config, load_labeled_data, load_default_validation_data

Commercial access denied and fallback to check for academic access.


In [5]:
data = load_default_validation_data()
# data = load_labeled_data("labeled_scenarios")

config_path = "env_configs/multi_agent/sac_training.yml"
config_from_yaml = OmegaConf.to_object(OmegaConf.load(config_path))
env_config = construct_env_config(config_from_yaml["env"])

In [6]:
env_config.render_mode="video"

In [7]:
env = gym.make('torchdriveenv-v0', args={'cfg': env_config, 'data': data})

INFO:torchdriveenv.gym_env:seed: 1035614716


In [8]:
def main(cfg, env):
    env.reset()
    while True:
        actions = torch.tensor([1, 0], dtype=torch.float32, device='cpu')  # accelerate hard without steering
        obs, reward, terminated, truncated, info = env.step(actions)  
        if terminated or truncated:
            break
    env.close()

In [9]:
main(env_config, env)

/home/kezhang/work/torchdriveenv/.venv/lib/python3.10/site-packages/gymnasium/core.py:311: UserWarning: WARN: env.torch_device to get variables from other wrappers is deprecated and will be removed in v1.0, to get this variable you can do `env.unwrapped.torch_device` for environment variables or `env.get_wrapper_attr('torch_device')` that will search the reminding wrappers.
  logger.warn(


### 